# Transformación, filtración y ordenamiento de datos

Hoy aprenderemos algunas técnicas para realizar transformación y reestructuración de datos.

La transformación de datos consiste en convertir un dato en otro dato utilizando algún tipo de proceso transformativo.

La reestructuración de datos tiene que ver con ver tu conjunto de datos desde diferentes perspectivas.

La transformación es muy útil para limpiar nuestro dataset y para dejarlo preparado para futuros análisis estadísticos, mientras que la reestructuración nos ayuda a entender mejor nuestro conjunto de datos y extraer información valiosa que pueda ser luego analizada o visualizada.

## Concatenación de Series



Cuando obtenemos nuestros datos en "cachitos", como cuando hacemos peticiones a una API, necesitamos luego unir todos nuestros datos en un solo DataFrame. Para eso podemos usar la función pd.concat de pandas. Primero vamos a aprender los principios básicos usando Series, para luego poder aplicar esos mismos principios a los DataFrames.

In [ ]:
import pandas as pd


In [ ]:
serie_1 = pd.Series([1, 2, 3], index=['a', 'b', 'c'])
serie_2 = pd.Series([4, 5, 6], index=['d', 'f', 'e'])

In [ ]:
pd.concat([serie_1, serie_2], axis=0)


También podemos concatenar horizontalmente:



In [ ]:
pd.concat([serie_1, serie_2], axis=1)


Podemos nombrar nuestras columnas para saber cuál era cuál:



In [ ]:
pd.concat([serie_1, serie_2], axis=1, keys=['serie_1', 'serie_2'])


Esto pasa si concatenamos horizontalmente usando el mismo índice:



In [ ]:
serie_3 = pd.Series([7, 8, 9], index=['a', 'b', 'c'])

pd.concat([serie_1, serie_3], axis=1, keys=['serie_1', 'serie_3'])

Si concatenamos verticalmente dos Series que comparten el índice, tenemos el problema de no poder diferenciar los índices:



In [ ]:
pd.concat([serie_1, serie_3], axis=0)


A veces queremos esto, pero cuando no, podemos agregar un segundo nivel de índice para hacer la diferencia:



In [ ]:
pd.concat([serie_1, serie_3], axis=0, keys=['serie_1', 'serie_3'])


Esto se llama un Multiíndice. Podemos acceder a un multiíndice en un solo nivel o en ambos:



In [ ]:
series_concat = pd.concat([serie_1, serie_3], axis=0, keys=['serie_1', 'serie_3'])


In [ ]:
series_concat.loc['serie_1']


In [ ]:
series_concat.loc[('serie_1', 'b')]


## Concatenación de DataFrames

Los mismos principios de concatenación aplican tanto a Series como a DataFrames. Vamos a verlos en acción y realizar una práctica para que nos quede todo súper claro.

In [ ]:
data_1 = {
    'column_1': [1, 2, 3],
    'column_2': [4, 5, 6]
}

df_1 = pd.DataFrame(data_1, index=['a', 'b', 'c'])

df_1

In [ ]:
data_2 = {
    'column_1': [7, 8, 9],
    'column_2': [10, 11, 12]
}

df_2 = pd.DataFrame(data_1, index=['d', 'e', 'f'])

df_2

In [ ]:
pd.concat([df_1, df_2], axis=0)


In [ ]:
pd.concat([df_1, df_2], axis=1)


In [ ]:
data_3 = {
    'column_3': [7, 8, 9],
    'column_4': [10, 11, 12]
}

df_3 = pd.DataFrame(data_3, index=['a', 'b', 'c'])

df_3

In [ ]:
pd.concat([df_1, df_3], axis=1)


Si concatenamos verticalmente con el mismo índice, no podemos diferenciarlos:



In [ ]:
data_4 = {
    'column_1': [7, 8, 9],
    'column_2': [10, 11, 12]
}

df_4 = pd.DataFrame(data_4, index=['a', 'b', 'c'])

df_4

In [ ]:
pd.concat([df_1, df_4], axis=0)


In [ ]:
df_concat = pd.concat([df_1, df_4], axis=0, keys=['df_1', 'df_4'])

df_concat

In [ ]:
df_concat.loc['df_1']


In [ ]:
df_concat.loc[('df_1', 'b')]


## Automatizando peticiones

In [ ]:
import pandas as pd
import requests
import time

Veamos cómo usar todo lo que aprendimos para automatizar el proceso de realizar múltiples peticiones a la API, reunirlas en un DataFrame

In [ ]:
endpoint = 'https://api.nasa.gov/neo/rest/v1/neo/browse/'
payload = {'api_key': 'tu_api_key_va_aqui'}

In [ ]:
dict_datos = {}

for i in range(0, 10):
    
    try:
        time.sleep(5)
        r = requests.get(endpoint, params=payload, timeout=5)

        if r.status_code == 200:
            json = r.json()

            data = json['near_earth_objects']
            dict_datos[i] = data

            new_link = json['links']['next']
            endpoint = new_link
    except:
        continue

In [ ]:
for key in dict_datos:
    normalized = pd.json_normalize(dict_datos[key])
    df = pd.DataFrame.from_dict(normalized)
    dict_datos[key] = df

In [ ]:
lista_de_dataframes = []

for key in dict_datos:
    lista_de_dataframes.append(dict_datos[key])

In [ ]:
df_completo = pd.concat(lista_de_dataframes, axis=0).reset_index(drop=True)


In [ ]:
df_completo


## CASTING

El primer tipo de transformación que veremos es el casting. Casting significa convertir un dato de un tipo de dato a otro tipo de dato. O sea, convertir una string a un int, un int a un float, un int a un datetime, etc.

Muchas veces los conjuntos de datos con los que nos topamos no tienen el formato adecuado o están muy sucios. Esto ocasiona que pandas no sepa cómo interpretar el tipo de datos con los que se enfrenta.

Veremos algunas técnicas parar hacer casting manualmente en los casos en los que pandas se equivoque o no sepa cómo proceder.



In [ ]:
import pandas as pd


In [ ]:
df = pd.read_csv('../../Datasets/new_york_times_bestsellers-dirty.csv', index_col=0)

df.head()


In [ ]:
df.dtypes


Específicamente, tenemos dos columnas con fechas (bestsellers_date.numberLong y published_date.numberLong) que tienen tipos object e int64. También tenemos una columna rank.numberInt que no tiene el tipo de dato adecuado.

Podemos usar el método astype para pasarle a nuestro DataFrame un diccionario de conversión. Por ejemplo, vamos a convertir nuestras dos columnas de fechas usando un diccionario de conversión. El tipo de dato que usamos para manejar fechas es el llamado datetime. Este tipo de dato nos permite manipular fechas y horarios muy eficientemente.

In [ ]:
diccionario_de_conversion = {
    'bestsellers_date.numberLong': 'datetime64[ns]',
    'published_date.numberLong': 'datetime64[ns]'
}


In [ ]:

temp = df.astype(diccionario_de_conversion)

temp.head()


In [ ]:
temp.dtypes


Como puedes ver, nuestras columnas han sido transformadas. Pero parece que hay un problema, puesto que hay muchísima diferencia de años entre la columna bestsellers_date y la columna published_date. Esto se debe a que published_date está en formato 'milisegundos desde La Época (la medianoche UTC del 1 de enero de 1970)' y pandas asume por default que estamos lidiando con nanosegundos.

Para evitar este problema vamos a usar el método pd.to_datetime para convertir published_date:

In [ ]:
pd.to_datetime(df['published_date.numberLong'], unit='ms')


to_datetime nos permite especificar las unidades para que la conversión se realice con éxito.

Vamos ahora qué pasa si queremos convertir rank.numberInt usando astype:

In [ ]:
df['rank.numberInt'].astype(int)


No podemos hacerlo porque hay unos valores tipo string que no pueden ser convertidos a int. Para esto usamos el método to_numeric, que nos permite indicar que cuando un error sea encontrado, debe de ser sustituido por un NaN:



In [ ]:
pd.to_numeric(df['rank.numberInt'], errors='coerce')


Vamos a reasignar el resultado al DataFrame original:



In [ ]:
df['rank.numberInt'] = pd.to_numeric(df['rank.numberInt'], errors='coerce')


Ahora, para convertirlo a tipo int podemos eliminar los NaNs y luego usar astype:



In [ ]:
df = df.dropna(axis=0).copy()


In [ ]:
df['rank.numberInt'] = df['rank.numberInt'].astype(int)


In [ ]:
df.dtypes


## Manipulación de strings

Manipular strings es todo un tema por sí mismo. Aprender a usar las herramientas de manipulación de strings es muy importante puesto que nos permite trabajar con datos no estructurados. Los datos no estructurados son básicamente secuencias de caracteres tipo texto.

Dado que en un texto las configuraciones, patrones y significados posibles son casi infinitos, necesitamos técnicas que nos ayuden a lidiar a mucho detalle con estos datos.

Para eso tenemos la propiedad str que estudiaremos a continuación.

Empecemos con la columna description que tiene un 'Descr:' al inicio de cada texto. Si queremos remover ese texto podemos usar el método replace de la propiedad str de esa Serie:

In [ ]:
df['description'].str.replace('Descr:', '')


In [ ]:
df['description'] = df['description'].str.replace('Descr:', '')


In [ ]:
df.loc[0, 'description']


Como puedes ver, tenemos también espacios vacíos al principio y final de nuestras strings. Vamos a removerlos usando strip:



In [ ]:
df['description'].str.strip()


In [ ]:
df['description'] = df['description'].str.strip()


In [ ]:
df.loc[0, 'description']


Ahora veamos la columna 'title', cuyos textos están en mayúsculas. Esto no es muy agradable, así que podemos usar algunos métodos para modificar el patrón de mayúsculas y minúsculas:

In [ ]:
df['title'].str.lower()


In [ ]:
df['title'].str.title()


In [ ]:
df['title'] = df['title'].str.title()


Ahora, digamos que queremos separar nuestra columna author en dos columnas author_first_name y author_last_name. Eso lo podemos hacer con el método split:

In [ ]:
df['author'].str.split(' ')


In [ ]:
df['author'].str.split(' ', expand=True)


In [ ]:
df[['author_first_name', 'author_last_name']] = df['author'].str.split(' ', expand=True)


In [ ]:
df.head()


## MAP

Otra cosa que podemos hacer es usar un mapeo de un dato a otro. Esto significa que le damos a pandas algún objeto que contenga una correspondencia entre un dato y otro para que realice una conversión.

map nos permite pasarle tanto un diccionario como una función para realizar la conversión de un dato a otro.

In [ ]:
df_2 = pd.read_csv('../../Datasets/new_york_times_bestsellers-dirty.csv', index_col=0)

df_2.head()


Digamos que queremos transformar los datos de nuestra columna 'rank.numberInt' para que el 'rankink' esté dado por letras, no por números.

Sabemos que hay un valor 'No Rank' en esa columna, así que nuestro diccionario de conversión podría verse así:



In [ ]:
int_a_letra = {
    '1': 'a',
    '2': 'b',
    '3': 'c',
    '4': 'd',
    '5': 'e',
    '6': 'f',
    '7': 'g',
    '8': 'h',
    '9': 'i',
    '10': 'j',
    '11': 'k',
    '12': 'l',
    '13': 'm',
    '14': 'n',
    '15': 'o',
    '16': 'p',
    'No Rank': 'z'
}


Lo aplicamos usando map:



In [ ]:
df['rank.numberInt'].map(int_a_letra).head(20)


También podemos usar una función para map. Por ejemplo esta función que realiza una correspondencia entre el precio de un libro y su representación en string:



In [ ]:
def double_to_money(value):
    
    return f'${value} USD'


In [ ]:
df['price.numberDouble'].map(double_to_money)


Lo único que tienes que pensar al usar map es: "¿Este dato tiene una correspondencia con otro dato que pueda representar con un diccionario o una función?". Y listo.



## APPLY

Otra manera de crear correspondencias es aplicando una función a nuestro DataFrame o Serie usando apply.

Para una Serie podemos usar apply para aplicar una función "elemento por elemento".

En DataFrames podemos usar este mismo método para aplicar funciones por filas o por columnas.



In [ ]:
def years_since_bestseller(value):
    
    as_datetime = pd.to_datetime(value, unit='ms')
    today = pd.to_datetime('today')
    difference_in_days = (today - as_datetime).days
    in_years = difference_in_days / 365
    
    return in_years


In [ ]:
df['published_date.numberLong'].apply(years_since_bestseller)


## FILTROS

Los filtros nos sirven para obtener subconjuntos de datos que tengan una cierta característica que necesitamos. Podemos "filtrar" solamente los datos que deseamos y dejar fuera datos indeseables.

Crear subconjuntos a partir de nuestro conjunto de datos es muy útil para entender mejor la conformación de nuestro dataset y para realizar análisis de muestras del total de nuestros datos.

¡Ésta es una de las herramientas que vas a estar usando más a menudo!

In [ ]:
df = pd.read_csv('../../Datasets/new_york_times_bestsellers-dirty.csv', index_col=0)

df.head()


Digamos que queremos todas los records donde el nombre del autor empiece con 'R'. Primero, usamos operadores de comparación (o en este caso, el método str.startswith) para obtener nuestro filtro:

In [ ]:
df['author'].str.startswith('R')

Lo que obtenemos de regreso es una Serie con la misma longitud que la Serie original. Se aplicó el método o la comparación a cada elemento de la Serie original. Estos métodos o comparaciones regresan True o False dependiendo de cada valor. La Serie resultante acumula los Trues y Falses que obtengamos de la comparación o de la aplicación del método.

Después, al pasar este filtro al operador de indexación del DataFrame, todas las filas a las que les corresponda un True se mantienen, mientras que las filas a las que les corresponde un False se dejan fuera del subconjunto resultante:

In [ ]:
df[df['author'].str.startswith('R')].head()


Podemos también guardar nuestros filtros en variables y después utilizarlos:



In [ ]:
filtro_precio_mayor_a_20 = df['price.numberDouble'] > 20


In [ ]:
df[filtro_precio_mayor_a_20].head()


Podemos incluso aplicar dos o más filtros utilizando operadores lógicos. En este caso, nuestro operador and se representa con un & y el operador or se representa con |:

In [ ]:
filtro_rank_numero_uno = df['rank.numberInt'] == '1'


In [ ]:
df[filtro_precio_mayor_a_20 & filtro_rank_numero_uno].head()
